In [20]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import librosa
import librosa.display

from scipy.signal import find_peaks 

import os

In [21]:
path = os.getcwd()

In [22]:
files = [p[:-4] for p in os.listdir(path+"/mv_plot")]

In [23]:
def moving_average(a, n=100):
    ret = np.nancumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    ret = ret[n - 1:] / n
    zero_arr = [0]*(n-1)
    return np.append(zero_arr, ret)

In [26]:
for file in files:
    sig, sr = librosa.load(path+"\\public_dataset\\"+file+".wav", sr=16000)
    
    fp = find_peaks(list(sig), height=0.2)[0] #높이가 최소 0.2 이상인 값을 찾음

    fp_diff = np.diff(fp)>3000 #다음 기침은 최소 0.2초 이후에 한다 가정
    fp_diff = np.append(fp_diff, False) #차분 값의 마지막에 False 값 추가
    if sum(fp_diff)==0: fp_index = 0
    else: fp_index = list(fp[fp_diff]) #True인 값 추출 (차이가 커지기 전 인덱스 추출)
    
    #한주기
    if fp_index==0:
        yt,idx = librosa.effects.trim(sig, top_db = 10)
        
        mv = moving_average(yt, 10)

        if sum(mv>0.2)!=0:
            mv_idx = list(mv>0.2).index(True)
            
            if len(yt[(mv_idx-100):])<1600: continue
            else:
                librosa.output.write_wav('./cut_wave/'+file+'.wav',
                                         yt[(mv_idx-100):], sr)
                
    else:
        yt,idx = librosa.effects.trim(sig[:fp_index[0]], top_db = 10)
        
        mv = moving_average(yt, 10)
        
        if sum(mv>0.2)!=0:
            mv_idx = list(mv>0.2).index(True)

            if len(yt[(mv_idx-100):])<1600: continue
            else:
                librosa.output.write_wav('./cut_wave/'+file+'.wav',
                                         yt[(mv_idx-100):], sr)